In [1]:
import pandas as pd
import numpy as np

In [2]:
book = pd.read_csv('Books.csv')
rating = pd.read_csv('Ratings.csv')       
user = pd.read_csv('Users.csv')

C:\Users\AMAN\AppData\Local\Temp\ipykernel_8532\1761757965.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('Books.csv')


In [3]:
book['Image-URL-M'] = book['Image-URL-M'].str.replace('http://', 'https://') 

In [4]:
print(book.shape)
print(user.shape)
print(rating.shape)

(271360, 8)
(278858, 3)
(1149780, 3)


In [5]:
files = [book,user,rating]
for i in files:

    print(i.isnull().sum())
    print('\n')

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


User-ID          0
Location         0
Age         110762
dtype: int64


User-ID        0
ISBN           0
Book-Rating    0
dtype: int64




In [6]:
for i in files:

    print(i.duplicated().sum())
    print('\n')

0


0


0




## Popularity based recommender

In [8]:
rating_with_book= rating.merge(book,on='ISBN')

In [9]:
num_rating_df = rating_with_book.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating' : 'num_ratings'},inplace=True)

In [10]:
avg_rating_df = rating_with_book.groupby('Book-Title')['Book-Rating'].mean().reset_index()  
avg_rating_df.rename(columns={'Book-Rating' : 'avg_ratings'},inplace=True)
avg_rating_df

,Book-Title,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [11]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_ratings
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [12]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_ratings',ascending = False).head(50)

In [13]:
popular_df = popular_df.merge(book,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Image-URL-M','num_ratings','avg_ratings','Book-Author']]

In [14]:
popular_df.head()

,Book-Title,Image-URL-M,num_ratings,avg_ratings,Book-Author
0,Harry Potter and the Prisoner of Azkaban (Book 3),https://images.amazon.com/images/P/0439136350....,428,5.852804,J. K. Rowling
3,Harry Potter and the Goblet of Fire (Book 4),https://images.amazon.com/images/P/0439139597....,387,5.824289,J. K. Rowling
5,Harry Potter and the Sorcerer's Stone (Book 1),https://images.amazon.com/images/P/0590353403....,278,5.737410,J. K. Rowling
9,Harry Potter and the Order of the Phoenix (Boo...,https://images.amazon.com/images/P/043935806X....,347,5.501441,J. K. Rowling
13,Harry Potter and the Chamber of Secrets (Book 2),https://images.amazon.com/images/P/0439064872....,556,5.183453,J. K. Rowling


## Collaborative recommender system

In [16]:
x = rating_with_book.groupby('User-ID').count()['Book-Rating']>200
padhe_likhe_user = x[x].index

In [17]:
filtered_rating = rating_with_book[rating_with_book['User-ID'].isin(padhe_likhe_user)]

In [18]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [19]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [20]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [21]:
pt.fillna(0,inplace=True)

In [22]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
similarity_score = cosine_similarity(pt)

In [25]:
similarity_score[1]

array([0.10255025, 1.        , 0.2364573 , 0.        , 0.1095379 ,
       0.10092909, 0.11786249, 0.18761978, 0.09658879, 0.04716095,
       0.09790994, 0.07830823, 0.19121955, 0.04828673, 0.11297907,
       0.11424415, 0.11290848, 0.08432401, 0.05619509, 0.09820417,
       0.17562977, 0.05779941, 0.15727218, 0.05719732, 0.11374482,
       0.        , 0.09515326, 0.08104912, 0.02894774, 0.07225874,
       0.01714222, 0.03288122, 0.04211669, 0.09380106, 0.04486288,
       0.25375719, 0.04682287, 0.20897363, 0.05826367, 0.30746648,
       0.05386542, 0.06892792, 0.04503831, 0.074284  , 0.13507508,
       0.05023148, 0.02267079, 0.06552582, 0.05644729, 0.06294081,
       0.07553841, 0.05578722, 0.        , 0.19483247, 0.07717041,
       0.01504361, 0.10744436, 0.05396449, 0.08539351, 0.10668897,
       0.11024468, 0.        , 0.02769053, 0.23980848, 0.08922627,
       0.04639526, 0.04350087, 0.09616002, 0.1153737 , 0.01531666,
       0.13588642, 0.03504529, 0.12520455, 0.12647968, 0.10989

In [26]:

def recommend(book_name):
    index = np.where(pt.index == book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:6]

    data = []
    for i in similar_items:
        item = []
        temp_df = book[book['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        data.append(item)

    return data
    

In [27]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'https://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'https://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'https://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'https://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg'],
 ['The Hours : A Novel',
  'Michael Cunningham',
  'https://images.amazon.com/images/P/0312243022.01.MZZZZZZZ.jpg']]

In [28]:
import pickle as pkl
pkl.dump(popular_df,open('popular.pkl','wb'))

In [29]:
pkl.dump(pt,open('pt.pkl','wb'))
pkl.dump(book,open('book.pkl','wb'))
pkl.dump(similarity_score,open('similarity_score.pkl','wb'))